In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow_addons as tfa
# Helper libraries
import numpy as np

print(tf.__version__)


In [ ]:
#https://www.tensorflow.org/guide/gpu
BATCH_SIZE = 1024
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)



In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile( 
                optimizer=tfa.optimizers.AdamW(learning_rate = 1e-3, 
                               weight_decay = 1e-5),
            #   tf.keras.optimizers.SGD(learning_rate=1e-3),#, weight_decay = 1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



In [ ]:
model.fit(train_images, 
            train_labels, 
            batch_size = BATCH_SIZE,
             validation_data=(test_images, test_labels),
            epochs=100)


In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)


In [ ]:
# hyperparameter tuning with optuna

import tensorflow as tf
import optuna

x_train = train_images
y_train = train_labels
x_test = test_images
y_test = test_labels

BATCH_SIZE = 1024

def create_model(learning_rate, weight_decay):
    # regularizer = tf.keras.regularizers.l2(weight_decay)
    model = tf.keras.models.Sequential([
        
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'), #, kernel_regularizer=regularizer),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax' ) # kernel_regularizer=regularizer)
    ])
    optimizer = tfa.optimizers.AdamW(learning_rate= learning_rate,
                                    weight_decay = weight_decay )
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1) ## Use suggest_float(..., log=True) instead ##
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3) #adjust ranges as needed.
    model = create_model(learning_rate, weight_decay)
    model.fit(x_train, y_train, epochs=3, validation_split=0.2, verbose=0)
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20) # Increase n_trials for more comprehensive search

print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial
print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

best_learning_rate = study.best_params['learning_rate']
best_weight_decay = study.best_params['weight_decay']

# Retrain with the best learning rate and weight decay.
best_model = create_model(best_learning_rate, best_weight_decay)
best_model.fit(x_train, y_train, epochs=10, validation_split=0.2)
final_loss, final_accuracy = best_model.evaluate(x_test,y_test)

print(f"Final Test Accuracy with best LR and WD: {final_accuracy}")

In [ ]:

import tensorflow as tf
import optuna

x_train = train_images
y_train = train_labels
x_test = test_images
y_test = test_labels

BATCH_SIZE = 2048

def create_model(learning_rate, weight_decay):
    # regularizer = tf.keras.regularizers.l2(weight_decay)
    model = tf.keras.models.Sequential([
        
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'), #, kernel_regularizer=regularizer),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax' ) # kernel_regularizer=regularizer)
    ])
    optimizer = tfa.optimizers.AdamW(learning_rate= learning_rate,
                                    weight_decay = weight_decay )
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 10) ## Use suggest_float(..., log=True) instead ##
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2) #adjust ranges as needed.
    model = create_model(learning_rate, weight_decay)
    model.fit(x_train, y_train, epochs=3, validation_split=0.2, verbose=0, batch_size = BATCH_SIZE)
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30) # Increase n_trials for more comprehensive search

print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial
print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

best_learning_rate = study.best_params['learning_rate']
best_weight_decay = study.best_params['weight_decay']

# Retrain with the best learning rate and weight decay.
best_model = create_model(best_learning_rate, best_weight_decay)
best_model.fit(x_train, 
y_train, 
epochs=10, 
validation_split=0.2,
batch_size = BATCH_SIZE)
final_loss, final_accuracy = best_model.evaluate(x_test,y_test)

print(f"Final Test Accuracy with best LR and WD: {final_accuracy}")

In [ ]:

# Retrain with the best learning rate and weight decay.
best_model = create_model(best_learning_rate, best_weight_decay)
best_model.fit(x_train, 
y_train, 
epochs=300, 
validation_split=0.2,
batch_size = BATCH_SIZE)
final_loss, final_accuracy = best_model.evaluate(x_test,y_test)

print(f"Final Test Accuracy with best LR and WD: {final_accuracy}")

In [ ]:
best_learning_rate, best_weight_decay